# Prerequisite

Following packages must be installed to run codes below

```bash
$ pip3 install openpyxl 
```


In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Add project path

In [5]:
import os
import sys
from pathlib import Path

PROJECT_PATH = Path().resolve().parent.parent
SRC_PATH = PROJECT_PATH / 'src'

sys.path.append(str(SRC_PATH))

In [1]:
exp_name = 'extended_beavers_model'

# Load data

In [6]:
from data_loader import SectorDataLoader

data_loader = SectorDataLoader()
data = (
    data_loader
    .load_dataset()
    .pipe(data_loader.add_beaver_indicator)
    .pipe(data_loader.add_label)
)

# Baseline classifier using Beaver's features

In [16]:
from project_paths import DATA_PATH

sectors = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]

beaver_features = [
    '유동자산/부채총계',
    '당기순이익(손실)/자산총계',
    '부채총계/자산총계',
    '순운전자본/자산총계',
    '유동부채/유동자산'
]

extended_financial_features = [
    '부채비율(%)',
    '매출액총이익률(%)',
    '매출액영업이익률(%)',
    '매출액순이익률(%)',
    '금융비용대매출액비율(%', 
    '금융비용대부채비율(%)', 
    '금융비용대총비용비율(%',
    '유보액/총자산(%)', 
    '유보액/납입자본(%)'
]

years_to_close = [
    'Closed_In_1Yr',
    'Closed_In_2Yrs',
]

x_data_year = list(range(2018, 2021))

EXP_RESULT_PATH = DATA_PATH / 'experiment_result' / exp_name

if not os.path.exists(EXP_RESULT_PATH) : 
    os.mkdir(EXP_RESULT_PATH)

In [23]:
import numpy as np
import pandas as pd
from pycaret.classification import ClassificationExperiment
from tqdm import tqdm

In [25]:
exp_result = None

for target_years_to_close in tqdm(years_to_close) : 
    for x_data_target_year in tqdm(x_data_year, leave=True) : 
        for sector in sectors : 

            filename = EXP_RESULT_PATH / f'Beaver_baseline_{sector}_{target_years_to_close}_{x_data_target_year}.csv'

            if not os.path.exists(filename) : 

                data_to_train = (
                    data
                    .loc[~data[beaver_features[0]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[1]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[2]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[3]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[4]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[lambda x : pd.to_datetime(x['결산년월'], format='%Y%m%d').dt.year==x_data_target_year]
                    .loc[lambda x : x['대분류']==sector]
                )

                exp = ClassificationExperiment()

                exp.setup(
                    (
                        data_to_train
                        .loc[:, beaver_features+extended_financial_features+[target_years_to_close]]
                        .reset_index(drop=True)
                    ),
                    target=target_years_to_close
                )

                models = exp.compare_models()

                result = (
                    exp.pull()
                    .assign(target=target_years_to_close)
                    .assign(x_data_yaer=x_data_target_year)
                    .assign(대분류=sector)
                    .reset_index(drop=False)
                    .rename(columns={'index':'model_name'})
                )
                result.to_csv(filename)

            else : 
                result = pd.read_csv(filename)

            if exp_result is None : 
                exp_result = result
            else : 
                exp_result = pd.concat([exp_result, result], axis=0)

  0%|                                                                                    | 0/3 [00:00<?, ?it/s]

,Description,Value
0,Session id,5341
1,Target,Closed_In_1Yr
2,Target type,Binary
3,Original data shape,"(1800, 15)"
4,Transformed data shape,"(1800, 15)"
5,Transformed train set shape,"(1259, 15)"
6,Transformed test set shape,"(541, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8777,0.5934,0.1085,0.7133,0.1827,0.1548,0.2398,0.0950
et,Extra Trees Classifier,0.8745,0.6159,0.1210,0.5817,0.1933,0.1587,0.2192,0.0680
lightgbm,Light Gradient Boosting Machine,0.8721,0.6308,0.1335,0.5369,0.2083,0.1675,0.2178,0.0530
qda,Quadratic Discriminant Analysis,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0100
dummy,Dummy Classifier,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0060
lda,Linear Discriminant Analysis,0.8674,0.5534,0.0000,0.0000,0.0000,-0.0044,-0.0085,0.0080
gbc,Gradient Boosting Classifier,0.8650,0.6321,0.0485,0.3483,0.0827,0.0552,0.0887,0.0810
ada,Ada Boost Classifier,0.8642,0.6217,0.0246,0.1333,0.0411,0.0217,0.0287,0.0320
ridge,Ridge Classifier,0.8594,0.0000,0.0125,0.0167,0.0143,-0.0034,-0.0092,0.0070
knn,K Neighbors Classifier,0.8515,0.5693,0.0485,0.1867,0.0764,0.0292,0.0355,0.0160


,Description,Value
0,Session id,4553
1,Target,Closed_In_1Yr
2,Target type,Binary
3,Original data shape,"(323, 15)"
4,Transformed data shape,"(323, 15)"
5,Transformed train set shape,"(226, 15)"
6,Transformed test set shape,"(97, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8006,0.6017,0.2350,0.6100,0.3107,0.2274,0.2790,0.0540
dummy,Dummy Classifier,0.7923,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0060
lr,Logistic Regression,0.7921,0.5197,0.0200,0.1000,0.0333,0.0203,0.0302,0.0300
lda,Linear Discriminant Analysis,0.7879,0.4631,0.0000,0.0000,0.0000,-0.0078,-0.0112,0.0070
et,Extra Trees Classifier,0.7875,0.6443,0.2900,0.4367,0.3330,0.2242,0.2327,0.0480
gbc,Gradient Boosting Classifier,0.7830,0.6028,0.2500,0.4233,0.3010,0.1965,0.2110,0.0230
svm,SVM - Linear Kernel,0.7565,0.0000,0.0000,0.0000,0.0000,-0.0561,-0.0668,0.0060
ada,Ada Boost Classifier,0.7565,0.5301,0.1700,0.3283,0.2111,0.0967,0.1083,0.0180
lightgbm,Light Gradient Boosting Machine,0.7522,0.5432,0.2750,0.4417,0.3196,0.1755,0.1941,0.0080
dt,Decision Tree Classifier,0.7340,0.6500,0.5200,0.4197,0.4465,0.2814,0.2943,0.0060


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0220
nb,Naive Bayes,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0100
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
rf,Random Forest Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0440
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0340
lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
dummy,Dummy Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070


,Description,Value
0,Session id,1338
1,Target,Closed_In_2Yrs
2,Target type,Multiclass
3,Target mapping,1: 0
4,Original data shape,"(11, 15)"
5,Transformed data shape,"(11, 15)"
6,Transformed train set shape,"(7, 15)"
7,Transformed test set shape,"(4, 15)"
8,Numeric features,14
9,Preprocess,True


,Description,Value
0,Session id,6139
1,Target,Closed_In_2Yrs
2,Target type,Multiclass
3,Target mapping,1: 0
4,Original data shape,"(100, 15)"
5,Transformed data shape,"(100, 15)"
6,Transformed train set shape,"(69, 15)"
7,Transformed test set shape,"(31, 15)"
8,Numeric features,14
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0220
nb,Naive Bayes,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
rf,Random Forest Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0430
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0350
lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
dummy,Dummy Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060



100%|████████████████████████████████████████████████████████████████████████████| 2/2 [02:21<00:00, 70.56s/it]


In [28]:
aggregated_result_path = EXP_RESULT_PATH / 'result.csv'

if not os.path.exists(aggregated_result_path) :
    exp_result.to_csv(aggregated_result_path)
    
exp_result

,model_name,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),target,x_data_yaer,대분류
0,rf,Random Forest Classifier,0.8777,0.5934,0.1085,0.7133,0.1827,0.1548,0.2398,0.095,Closed_In_1Yr,2018,제조업(10~34)
1,et,Extra Trees Classifier,0.8745,0.6159,0.1210,0.5817,0.1933,0.1587,0.2192,0.068,Closed_In_1Yr,2018,제조업(10~34)
2,lightgbm,Light Gradient Boosting Machine,0.8721,0.6308,0.1335,0.5369,0.2083,0.1675,0.2178,0.053,Closed_In_1Yr,2018,제조업(10~34)
3,qda,Quadratic Discriminant Analysis,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.010,Closed_In_1Yr,2018,제조업(10~34)
4,dummy,Dummy Classifier,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.006,Closed_In_1Yr,2018,제조업(10~34)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.007,Closed_In_2Yrs,2020,건설업(41~42)
6,lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.006,Closed_In_2Yrs,2020,건설업(41~42)
7,et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.035,Closed_In_2Yrs,2020,건설업(41~42)
8,lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.006,Closed_In_2Yrs,2020,건설업(41~42)
